<a href="https://colab.research.google.com/github/financieras/ai/blob/main/logistic_regression/jupyter/prediccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regession
Proceso completo de entrenamiento con datos de 'train' y predicción con datos de 'test'.

En este desarrollo faltan la estadística descriptiva y los gráficos.

## Trabajar con los datos de entrenamiento
Partimos de un archivo CSV y lo limpiamos, seleccionamos columnas y normalizamos.

- Los archivos de origen están en Drive y son:
    1. 'datasets/dataset_train.csv'
    2. 'datasets/dataset_test.csv'
- Los archivos están en Google Drive.  
- En esta primera fase vamos a trabajar solo con los datos de entrenamiento.  
- Leemos el archivo CSV y construimos con él un DataFrame.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Ruta al archivo en Google Drive
input_file = '/content/drive/My Drive/datasets/dataset_train.csv'

# Leer el archivo CSV y crear el DataFrame
df = pd.read_csv(input_file)

# Mostrar información sobre las columnas del DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          1600 non-null   int64  
 1   Hogwarts House                 1600 non-null   object 
 2   First Name                     1600 non-null   object 
 3   Last Name                      1600 non-null   object 
 4   Birthday                       1600 non-null   object 
 5   Best Hand                      1600 non-null   object 
 6   Arithmancy                     1566 non-null   float64
 7   Astronomy                      1568 non-null   float64
 8   Herbology                      1567 non-null   float64
 9   Defense Against the Dark Arts  1569 non-null   float64
 10  Divination                     1561 non-null   float64
 11  Muggle Studies                 1565 non-null   float64
 12  Ancient Runes                  1565 non-null   f

In [3]:
# Remove duplicate rows
df = df.drop_duplicates()

# Convert 'Best Hand' to a binary variable (0 for Left, 1 for Right)
df['Best Hand'] = df['Best Hand'].map({'Left': 0, 'Right': 1})

# Calculate age based on the maximum date in Birthday column
df['Birthday'] = pd.to_datetime(df['Birthday'])
reference_date = df['Birthday'].max()
df['Age'] = (reference_date - df['Birthday']).dt.days / 365.25

# Remove unnecessary columns
columns_to_drop = ['First Name', 'Last Name', 'Birthday']
df = df.drop(columns=columns_to_drop)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          1600 non-null   int64  
 1   Hogwarts House                 1600 non-null   object 
 2   Best Hand                      1600 non-null   int64  
 3   Arithmancy                     1566 non-null   float64
 4   Astronomy                      1568 non-null   float64
 5   Herbology                      1567 non-null   float64
 6   Defense Against the Dark Arts  1569 non-null   float64
 7   Divination                     1561 non-null   float64
 8   Muggle Studies                 1565 non-null   float64
 9   Ancient Runes                  1565 non-null   float64
 10  History of Magic               1557 non-null   float64
 11  Transfiguration                1566 non-null   float64
 12  Potions                        1570 non-null   f

In [5]:
df.head()

,Index,Hogwarts House,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Age
0,0,Ravenclaw,0,58384.0,-487.886086,5.727180,4.878861,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,0.715939,-232.79405,-26.89,1.598905
1,1,Slytherin,1,67239.0,-552.060507,-5.987446,5.520605,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,0.091674,-252.18425,-113.45,2.058864
2,2,Ravenclaw,0,23702.0,-366.076117,7.725017,3.660761,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-0.515327,-227.34265,30.42,2.004107
3,3,Gryffindor,0,32667.0,697.742809,-6.497214,-6.977428,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-0.014040,-256.84675,200.64,1.210130
4,4,Gryffindor,0,60158.0,436.775204,-7.820623,NaN,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-0.264070,-256.38730,157.98,3.104723


## Imputación de datos faltantes usando 'Astronomy'
Usamos la correlación perfecta r=-1 entre 'Astronomy' y 'Defense Against the Dark Arts' para realizar la imputación de los datos faltantes que se puedan imputar de 'Defense Against the Dark Arts' aprovechando esta característica.

Lo único que hay que hacer es tomar el dato de 'Astronomy' y multiplicar por -0.01 para lograr conseguir el dato faltante de  'Defense Against the Dark Arts'.

In [6]:
# Imputar Defense Against the Dark Arts usando Astronomy cuando sea posible
# 1. Crear máscara para identificar dónde hay valores nulos en Defense y datos en Astronomy
mask = df['Defense Against the Dark Arts'].isna() & df['Astronomy'].notna()

# 2. Imputar los valores usando la relación perfecta (Astronomy * -0.01)
df.loc[mask, 'Defense Against the Dark Arts'] = df.loc[mask, 'Astronomy'] * -0.01

# 3. Verificar el resultado
print("Valores no nulos en Defense Against the Dark Arts:")
print(f"Antes de imputación: 1569/1600")
print(f"Después de imputación: {df['Defense Against the Dark Arts'].count()}/1600")

Valores no nulos en Defense Against the Dark Arts:
Antes de imputación: 1569/1600
Después de imputación: 1600/1600


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          1600 non-null   int64  
 1   Hogwarts House                 1600 non-null   object 
 2   Best Hand                      1600 non-null   int64  
 3   Arithmancy                     1566 non-null   float64
 4   Astronomy                      1568 non-null   float64
 5   Herbology                      1567 non-null   float64
 6   Defense Against the Dark Arts  1600 non-null   float64
 7   Divination                     1561 non-null   float64
 8   Muggle Studies                 1565 non-null   float64
 9   Ancient Runes                  1565 non-null   float64
 10  History of Magic               1557 non-null   float64
 11  Transfiguration                1566 non-null   float64
 12  Potions                        1570 non-null   f

In [8]:
df.head()

,Index,Hogwarts House,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Age
0,0,Ravenclaw,0,58384.0,-487.886086,5.727180,4.878861,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,0.715939,-232.79405,-26.89,1.598905
1,1,Slytherin,1,67239.0,-552.060507,-5.987446,5.520605,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,0.091674,-252.18425,-113.45,2.058864
2,2,Ravenclaw,0,23702.0,-366.076117,7.725017,3.660761,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-0.515327,-227.34265,30.42,2.004107
3,3,Gryffindor,0,32667.0,697.742809,-6.497214,-6.977428,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-0.014040,-256.84675,200.64,1.210130
4,4,Gryffindor,0,60158.0,436.775204,-7.820623,-4.367752,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-0.264070,-256.38730,157.98,3.104723


## Normalización
Es crucial usar EXACTAMENTE los mismos parámetros de normalización (media y desviación estándar) que se usen para los datos de entrenamiento cuando se aplique la normalización a los datos de test. No se deben calcular nuevos parámetros con los datos de test.

In [9]:
# Select numerical columns to normalize (including Age which is already float64)
columns_to_normalize = df.select_dtypes(include=['float64']).columns.tolist()

# Function to normalize using mean and standard deviation
def normalize(column, mean=None, std=None):
    """
    Normaliza una columna usando media y desviación estándar

    Parámetros:
    column: columna a normalizar
    mean: media preexistente (opcional)
    std: desviación estándar preexistente (opcional)

    Retorna:
    Si mean y std son None:
        column_normalized, mean, std
    Si mean y std son proporcionados:
        column_normalized
    """
    if mean is None or std is None:
        mean = column.mean()
        std = column.std()
        return (column - mean) / std, mean, std
    return (column - mean) / std

# Diccionario para guardar los parámetros de normalización
normalization_params = {}

# Aplicar normalización y guardar parámetros
for column in columns_to_normalize:
    df[column], mean, std = normalize(df[column])
    normalization_params[column] = {'mean': mean, 'std': std}

# Apply one-hot encoding for Hogwarts House
df = pd.get_dummies(df, columns=['Hogwarts House'], prefix='House', dtype=float)

# Convert int values in 'Best Hand' column to float64 values
df['Best Hand'] = df['Best Hand'].astype(float)


print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          1600 non-null   int64  
 1   Best Hand                      1600 non-null   float64
 2   Arithmancy                     1566 non-null   float64
 3   Astronomy                      1568 non-null   float64
 4   Herbology                      1567 non-null   float64
 5   Defense Against the Dark Arts  1600 non-null   float64
 6   Divination                     1561 non-null   float64
 7   Muggle Studies                 1565 non-null   float64
 8   Ancient Runes                  1565 non-null   float64
 9   History of Magic               1557 non-null   float64
 10  Transfiguration                1566 non-null   float64
 11  Potions                        1570 non-null   float64
 12  Care of Magical Creatures      1560 non-null   f

In [10]:
df.head()

,Index,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Age,House_Gryffindor,House_Hufflepuff,House_Ravenclaw,House_Slytherin
0,0,0.0,0.524552,-1.014194,0.878628,1.010516,0.377371,1.021139,0.345639,0.512444,0.219633,-0.686183,0.791972,1.204553,-0.500330,-0.653126,0.0,0.0,1.0,0.0
1,1,1.0,1.055434,-1.137535,-1.365690,1.133689,-2.109573,-0.540256,-1.204191,0.258503,0.653769,0.412462,0.149365,-1.002983,-1.386928,-0.331503,0.0,0.0,0.0,1.0
2,2,0.0,-1.554729,-0.780078,1.261379,0.776719,0.718622,1.828915,1.005195,0.133871,1.314249,0.882556,-0.475471,1.825184,0.086673,-0.369792,0.0,0.0,1.0,0.0
3,3,0.0,-1.017252,1.264555,-1.463352,-1.265132,0.209874,-0.642366,0.265645,-1.756242,-2.486237,-1.629193,0.040544,-1.533799,1.830165,-0.924974,1.0,0.0,0.0,0.0
4,4,0.0,0.630908,0.762982,-1.716894,-0.764241,-0.220901,-0.451681,0.974516,-1.447763,-2.099988,-0.520770,-0.216832,-1.481492,1.393217,0.399805,1.0,0.0,0.0,0.0


In [11]:
# Remove columns
reduce_columns = ['Arithmancy', 'Astronomy', 'Divination', 'Muggle Studies', 'Ancient Runes', 'History of Magic', 'Transfiguration', 'Care of Magical Creatures']
df = df.drop(columns=reduce_columns)

# Drop rows with missing data
df = df.dropna()

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, 0 to 1599
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          1537 non-null   int64  
 1   Best Hand                      1537 non-null   float64
 2   Herbology                      1537 non-null   float64
 3   Defense Against the Dark Arts  1537 non-null   float64
 4   Potions                        1537 non-null   float64
 5   Charms                         1537 non-null   float64
 6   Flying                         1537 non-null   float64
 7   Age                            1537 non-null   float64
 8   House_Gryffindor               1537 non-null   float64
 9   House_Hufflepuff               1537 non-null   float64
 10  House_Ravenclaw                1537 non-null   float64
 11  House_Slytherin                1537 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 156.1 KB
None


In [12]:
df.head()

,Index,Best Hand,Herbology,Defense Against the Dark Arts,Potions,Charms,Flying,Age,House_Gryffindor,House_Hufflepuff,House_Ravenclaw,House_Slytherin
0,0,0.0,0.878628,1.010516,-0.686183,1.204553,-0.500330,-0.653126,0.0,0.0,1.0,0.0
1,1,1.0,-1.365690,1.133689,0.412462,-1.002983,-1.386928,-0.331503,0.0,0.0,0.0,1.0
2,2,0.0,1.261379,0.776719,0.882556,1.825184,0.086673,-0.369792,0.0,0.0,1.0,0.0
3,3,0.0,-1.463352,-1.265132,-1.629193,-1.533799,1.830165,-0.924974,1.0,0.0,0.0,0.0
4,4,0.0,-1.716894,-0.764241,-0.520770,-1.481492,1.393217,0.399805,1.0,0.0,0.0,0.0


## Variables del modelo

In [13]:
import numpy as np

# 1. Crear la matriz y con todas las casas ya que estoy en un caso de clasificación multinomial
y = df[['House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']]

# 2. Seleccionar las características (X)
X = df[['Best Hand', 'Age', 'Herbology', 'Defense Against the Dark Arts',
        'Potions', 'Charms', 'Flying']]

# 3. Agregar columna de 1's para el término de sesgo (bias)
X = np.c_[np.ones(len(X)), X]

# Convertir a arrays de numpy para operaciones más eficientes
X = np.array(X)
y = np.array(y)

print("\nPrimeros 3 valores de X:")
print(X[:3])

print("\nPrimeros 3 valores de y:")
print(y[:3])


Primeros 3 valores de X:
[[ 1.          0.         -0.65312594  0.87862839  1.01051558 -0.68618295
   1.20455291 -0.50032993]
 [ 1.          1.         -0.33150337 -1.36568958  1.13368932  0.41246159
  -1.00298294 -1.3869281 ]
 [ 1.          0.         -0.36979177  1.26137897  0.77671855  0.88255609
   1.82518401  0.08667263]]

Primeros 3 valores de y:
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


## Función Sofmax
- Para un caso multinomial, en lugar de la función sigmoide, es recomendado usar la función softmax.  
- La función sigmoide se usa principalmente para clasificación binaria, mientras que softmax es la generalización para múltiples clases.
- La función softmax se define como:
- softmax(z)_i = exp(z_i) / Σ(exp(z_j))

In [14]:
def softmax(z):
    """
    Calcula la función softmax para clasificación multinomial

    Parámetros:
    z: matriz de forma (n_muestras, n_clases)

    Retorna:
    matriz de probabilidades de forma (n_muestras, n_clases)
    donde cada fila suma 1
    """
    # Restamos el máximo para estabilidad numérica
    # Esto evita desbordamiento en exp() con números grandes
    z_shifted = z - np.max(z, axis=1, keepdims=True)

    # Calculamos exp() de los valores desplazados
    exp_scores = np.exp(z_shifted)

    # Normalizamos dividiendo por la suma
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

# Podemos probar la función con algunos valores
test_values = np.array([
    [1, 2, 3, 4],
    [2, 1, 0, -1],
    [0, 0, 0, 0]
])
print("Valores de prueba:\n", test_values)
print("\nProbabilidades softmax:\n", softmax(test_values))
print("\nVerificar que cada fila suma 1:\n", np.sum(softmax(test_values), axis=1))

Valores de prueba:
 [[ 1  2  3  4]
 [ 2  1  0 -1]
 [ 0  0  0  0]]

Probabilidades softmax:
 [[0.0320586  0.08714432 0.23688282 0.64391426]
 [0.64391426 0.23688282 0.08714432 0.0320586 ]
 [0.25       0.25       0.25       0.25      ]]

Verificar que cada fila suma 1:
 [1. 1. 1.]


## Función de pérdida
- Para el caso multinomial, necesitamos adaptar la función de pérdida para manejar múltiples clases.
- La función de pérdida logarítmica multinomial también se llama cross-entropy.

In [15]:
def compute_cost(X, y, W):
    """
    Calcula la función de pérdida logarítmica (cross-entropy) para clasificación multinomial

    Parámetros:
    X: matriz de características (incluyendo columna de 1's) de forma (n_muestras, n_características)
    y: matriz one-hot de etiquetas reales de forma (n_muestras, n_clases)
    W: matriz de pesos de forma (n_características, n_clases)

    Retorna:
    J: valor de la función de pérdida
    """
    m = X.shape[0]  # número de muestras

    # Calcular predicciones
    z = np.dot(X, W)  # (n_muestras, n_clases)
    h = softmax(z)    # (n_muestras, n_clases)

    # Calcular pérdida logarítmica
    epsilon = 1e-15  # para evitar log(0)

    # Multiplicación elemento a elemento de y real con log de predicciones
    # y sumamos sobre todas las clases (axis=1) y todas las muestras
    J = -(1/m) * np.sum(y * np.log(h + epsilon))

    return J

# Ejemplo de uso:
# Supongamos que tenemos:
# X: (1508, 8) - 1508 muestras, 7 características + 1 columna de unos
# y: (1508, 4) - etiquetas one-hot para 4 casas
# W: (8, 4) - pesos para cada característica y cada clase

# Inicializar pesos aleatoriamente
W = np.random.randn(X.shape[1], y.shape[1]) * 0.01

# Calcular coste inicial
cost = compute_cost(X, y, W)
print(f"Coste inicial: {cost}")

Coste inicial: 1.3991035715550122


Las principales diferencias con la versión binaria son:
- Usamos W (matriz de pesos) en lugar de theta (vector)
- Usamos softmax en lugar de sigmoid
- La fórmula de la pérdida es más simple porque y es one-hot encoding (solo el término positivo importa)
- No necesitamos el término (1-y) porque las etiquetas ya están en formato one-hot

Esta función de pérdida penalizará más cuando el modelo asigne probabilidades bajas a las clases correctas y nos servirá para entrenar el modelo mediante descenso por gradiente.

## Descenso del Gradiente
Para el caso multinomial, necesitamos adaptar la función de descenso del gradiente para manejar la matriz de pesos W en lugar del vector theta.

In [16]:
def gradient_descent_multinomial(X, y, learning_rate=0.1, num_iterations=1000, epsilon=1e-8):
    """
    Implementa el descenso del gradiente para regresión logística multinomial

    Parámetros:
    X: matriz de características (incluyendo columna de 1's) de forma (n_muestras, n_características)
    y: matriz one-hot de etiquetas reales de forma (n_muestras, n_clases)
    learning_rate: tasa de aprendizaje (alpha)
    num_iterations: número máximo de iteraciones
    epsilon: umbral para early stopping

    Retorna:
    W: matriz de pesos optimizada
    cost_history: lista con el valor de la función de pérdida en cada iteración
    """
    # Inicializar matriz de pesos W con valores pequeños aleatorios
    n_features = X.shape[1]
    n_classes = y.shape[1]
    W = np.random.randn(n_features, n_classes) * 0.01

    # Lista para guardar el historial de costes
    cost_history = []

    # Número de muestras
    m = X.shape[0]

    # Calcular coste inicial
    prev_cost = compute_cost(X, y, W)
    cost_history.append(prev_cost)

    # Descenso del gradiente
    for i in range(num_iterations):
        # Calcular predicciones actuales
        z = np.dot(X, W)
        h = softmax(z)

        # Calcular gradiente
        # El gradiente es (1/m) * X^T * (h - y)
        gradient = (1/m) * np.dot(X.T, (h - y))

        # Actualizar pesos
        W = W - learning_rate * gradient

        # Calcular nuevo coste
        current_cost = compute_cost(X, y, W)
        cost_history.append(current_cost)

        # Imprimir progreso cada 1000 iteraciones
        if i % 1000 == 0:
            print(f'Iteración {i}: Coste = {current_cost}')

        # Early stopping
        if abs(prev_cost - current_cost) < epsilon:
            print(f'\nConvergencia alcanzada en la iteración {i}')
            print(f'Diferencia en coste: {abs(prev_cost - current_cost)}')
            break

        prev_cost = current_cost

    return W, cost_history

# Ejemplo de uso:
# W, cost_history = gradient_descent_multinomial(X, y)

Las principales diferencias con la versión binomial son:
- Inicializamos una matriz W en lugar de un vector theta
- El gradiente se calcula usando la fórmula para multinomial: (1/m) * X^T * (h - y)
    - h son las probabilidades softmax
    - La resta (h - y) se hace sobre todas las clases

No necesitamos una función compute_gradient separada, ya que el cálculo es más directo
Todas las operaciones matriciales están adaptadas para manejar múltiples clases

Para hacer predicciones con el modelo entrenado, podrías usar:

In [17]:
def predict(X, W):
    """
    Realiza predicciones usando los pesos aprendidos

    Parámetros:
    X: matriz de características (incluyendo columna de 1's)
    W: matriz de pesos optimizada

    Retorna:
    predicciones: matriz de probabilidades para cada clase
    """
    z = np.dot(X, W)
    return softmax(z)

podemos ejecutarlo así:

In [ ]:
# Hiperparámetros
learning_rate = 0.1
num_iterations = 5_000

# Ejecutar el descenso del gradiente multinomial
W_optimal, cost_history = gradient_descent_multinomial(X, y, learning_rate, num_iterations)

# Imprimir coste inicial y final
print(f"Coste inicial: {cost_history[0]}")
print(f"Coste final: {cost_history[-1]}")

# Hacer predicciones
probabilities = predict(X, W_optimal)

# Obtener la clase predicha
predicted_houses = np.argmax(probabilities, axis=1)
actual_houses = np.argmax(y, axis=1)
accuracy = np.mean(predicted_houses == actual_houses)
print(f"\nPrecisión del modelo: {accuracy:.4f}")

# Visualizar la evolución del coste
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(cost_history)
plt.title('Evolución de la función de coste')
plt.xlabel('Iteraciones')
plt.ylabel('Coste')
plt.grid(True)
plt.show()

# Tratamiento de los datos de test

Primero cargar los datos de test:

In [19]:
import pandas as pd

# Ruta al archivo en Google Drive para test
input_file_test = '/content/drive/My Drive/datasets/dataset_test.csv'

# Leer el archivo CSV y crear el DataFrame de test
df_test = pd.read_csv(input_file_test)

print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          400 non-null    int64  
 1   Hogwarts House                 0 non-null      float64
 2   First Name                     400 non-null    object 
 3   Last Name                      400 non-null    object 
 4   Birthday                       400 non-null    object 
 5   Best Hand                      400 non-null    object 
 6   Arithmancy                     387 non-null    float64
 7   Astronomy                      387 non-null    float64
 8   Herbology                      389 non-null    float64
 9   Defense Against the Dark Arts  392 non-null    float64
 10  Divination                     394 non-null    float64
 11  Muggle Studies                 390 non-null    float64
 12  Ancient Runes                  392 non-null    flo

## Procesamiento de los datos de `test`
Comentario para la posterior: Imputación de datos faltantes

Vamos a tratar los datos de test igual que los datos de entrenamiento.

La imputación de los datos faltantes se hará usando regresión lineal múltiple.

El caso de 'Astronomy' es importante ya que mantiene con 'Defense Against the Dark Arts' una correlación perfecta con r=-1, lo cual nos ayudará a imputar bastantes datos con precisión total. Simplemnete dividiendo entre -100 los catos de 'Astronomy' se obtienen los datos de 'Defense Against the Dark Arts'.



In [20]:
# Remove duplicate rows
df_test = df_test.drop_duplicates()

# Convert 'Best Hand' to a binary variable (0 for Left, 1 for Right)
df_test['Best Hand'] = df_test['Best Hand'].map({'Left': 0, 'Right': 1})

# Calculate age using the SAME reference date from training
df_test['Birthday'] = pd.to_datetime(df_test['Birthday'])
# no calculamos una nueva variable 'reference_date' usamos la de los datos de entrenamiento
df_test['Age'] = (reference_date - df_test['Birthday']).dt.days / 365.25

# Remove unnecessary columns
columns_to_drop = ['First Name', 'Last Name', 'Birthday', 'Hogwarts House']
df_test = df_test.drop(columns=columns_to_drop)

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          400 non-null    int64  
 1   Best Hand                      400 non-null    int64  
 2   Arithmancy                     387 non-null    float64
 3   Astronomy                      387 non-null    float64
 4   Herbology                      389 non-null    float64
 5   Defense Against the Dark Arts  392 non-null    float64
 6   Divination                     394 non-null    float64
 7   Muggle Studies                 390 non-null    float64
 8   Ancient Runes                  392 non-null    float64
 9   History of Magic               389 non-null    float64
 10  Transfiguration                389 non-null    float64
 11  Potions                        390 non-null    float64
 12  Care of Magical Creatures      392 non-null    flo

## Rellenar datos faltantes usando la correlación perfecta con 'Astronomy'

In [21]:
# Imputar Defense Against the Dark Arts usando Astronomy cuando sea posible
# 1. Crear máscara para identificar dónde hay valores nulos en Defense y datos en Astronomy
mask = df_test['Defense Against the Dark Arts'].isna() & df_test['Astronomy'].notna()

# 2. Imputar los valores usando la relación perfecta (Astronomy * -0.01)
df_test.loc[mask, 'Defense Against the Dark Arts'] = df_test.loc[mask, 'Astronomy'] * -0.01

# 3. Verificar el resultado
print("\nValores no nulos en Defense Against the Dark Arts (datos de test):")
print(f"Antes de imputación: 392/400")
print(f"Después de imputación: {df_test['Defense Against the Dark Arts'].count()}/400")


Valores no nulos en Defense Against the Dark Arts (datos de test):
Antes de imputación: 392/400
Después de imputación: 399/400


In [22]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Index                          400 non-null    int64  
 1   Best Hand                      400 non-null    int64  
 2   Arithmancy                     387 non-null    float64
 3   Astronomy                      387 non-null    float64
 4   Herbology                      389 non-null    float64
 5   Defense Against the Dark Arts  399 non-null    float64
 6   Divination                     394 non-null    float64
 7   Muggle Studies                 390 non-null    float64
 8   Ancient Runes                  392 non-null    float64
 9   History of Magic               389 non-null    float64
 10  Transfiguration                389 non-null    float64
 11  Potions                        390 non-null    float64
 12  Care of Magical Creatures      392 non-null    flo

## Imputación de datos faltantes con KNN usando solo filas completas

### 1. Preparación de los datos:

In [23]:
# Identificar filas completas (sin valores faltantes)
complete_rows = df_test.dropna()

# Identificar filas con valores faltantes
incomplete_rows = df_test[df_test.isnull().any(axis=1)]

### 2. Función para calcular distancia euclidiana:

In [24]:
import numpy as np

def euclidean_distance(row1, row2, columns):
    """
    Calcula la distancia euclidiana entre dos filas usando solo las columnas especificadas
    """
    return np.sqrt(sum((row1[columns] - row2[columns])**2))

### 3. Función para encontrar k vecinos más cercanos:

In [25]:
def get_k_neighbors(target_row, complete_data, columns_to_use, k=5):
    """
    Encuentra los k vecinos más cercanos para una fila dada
    """
    distances = []
    for idx, row in complete_data.iterrows():
        dist = euclidean_distance(target_row, row, columns_to_use)
        distances.append((idx, dist))

    # Ordenar por distancia y obtener los k más cercanos
    distances.sort(key=lambda x: x[1])
    return [idx for idx, _ in distances[:k]]

### 4. Función de imputación:

In [26]:
def impute_missing_values(df, k=5):
    """
    Imputa valores faltantes usando KNN
    """
    df_imputed = df.copy()
    complete_data = df.dropna()

    # Para cada fila con valores faltantes
    for idx, row in df[df.isnull().any(axis=1)].iterrows():
        # Identificar columnas con valores faltantes en esta fila
        missing_cols = row[row.isnull()].index

        # Identificar columnas disponibles para calcular distancias
        available_cols = row[row.notnull()].index

        # Encontrar k vecinos más cercanos
        neighbors_idx = get_k_neighbors(row, complete_data, available_cols, k)

        # Imputar cada columna faltante con la media de los vecinos
        for col in missing_cols:
            neighbor_values = complete_data.loc[neighbors_idx, col]
            df_imputed.loc[idx, col] = neighbor_values.mean()

    return df_imputed

### 5. Aplicación del proceso:

In [27]:
# Aplicar la imputación
df_test_imputed = impute_missing_values(df_test, k=5)

# Verificar resultados
print("\nVerificación de valores faltantes después de imputación:")
print(df_test_imputed.isnull().sum())


Verificación de valores faltantes después de imputación:
Index                            0
Best Hand                        0
Arithmancy                       0
Astronomy                        0
Herbology                        0
Defense Against the Dark Arts    0
Divination                       0
Muggle Studies                   0
Ancient Runes                    0
History of Magic                 0
Transfiguration                  0
Potions                          0
Care of Magical Creatures        0
Charms                           0
Flying                           0
Age                              0
dtype: int64


## Normalización de los datos de test
- Normalizar los datos de test usando los parámetros (media y desviación estándar) que guardamos del conjunto de entrenamiento.
- Esto es crucial porque debemos aplicar exactamente la misma transformación que aplicamos a los datos de entrenamiento.

In [28]:
# Convertir Best Hand a float64 como hicimos en training
df_test['Best Hand'] = df_test['Best Hand'].astype(float)

# Normalizar usando los parámetros guardados de training
for column in df_test.columns:
    if column in normalization_params:
        mean = normalization_params[column]['mean']
        std = normalization_params[column]['std']
        df_test[column] = normalize(df_test[column], mean, std)

print("\nPrimeros 3 registros después de la normalización:")
df_test.head(5)


Primeros 3 registros después de la normalización:


,Index,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Age
0,0,1.0,-0.479176,1.261390,0.360013,-1.261971,1.165280,-0.289017,-0.965420,0.281215,0.377241,-0.724744,0.439834,-0.126065,-0.364411,-1.715629
1,1,0.0,-0.256752,-0.789243,0.349486,0.785871,0.768919,1.536298,1.007714,0.790127,0.406955,0.814602,0.649395,1.375558,-0.493877,0.901383
2,2,0.0,-0.367664,0.539127,-1.403671,-0.540691,0.352583,-0.833736,0.868643,-1.927101,-2.122591,-1.302330,-1.640549,-1.095106,1.825147,0.422778
3,3,1.0,0.731209,0.706145,0.732118,-0.707481,NaN,-0.461791,-0.640240,NaN,0.301433,-0.411010,1.253926,-0.073112,-0.339112,-1.430380
4,4,1.0,NaN,0.477689,0.497470,-0.479336,0.416839,-0.462927,-1.035283,-0.019601,0.482287,-1.016498,0.160101,-0.071001,-0.297834,0.156674


## Preparar los datos de test
Preparar X_test seleccionando solo las columnas usadas en entrenamiento y añadiendo el término de sesgo.

Necesitaremos preparar los datos en el formato correcto para hacer las predicciones:

In [29]:
# Preparar X_test (agregar columna de 1's para el término de sesgo)
X_test = df_test[['Best Hand', 'Age', 'Herbology', 'Defense Against the Dark Arts',
                  'Potions', 'Charms', 'Flying']]
X_test = np.c_[np.ones(len(X_test)), X_test]
X_test = np.array(X_test)

print("\nForma de X_test:", X_test.shape)


Forma de X_test: (400, 8)


## Predicciones con los datos de test
Ahora podemos usar la función predict que ya teníamos y los pesos óptimos (W_optimal) para hacer las predicciones.

In [30]:
# Hacer predicciones usando los pesos óptimos del entrenamiento
probabilities = predict(X_test, W_optimal)

# Obtener la clase predicha (la casa con mayor probabilidad)
predicted_houses = np.argmax(probabilities, axis=1)

# Convertir los índices a nombres de casas
house_names = ['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']
predictions = [house_names[idx] for idx in predicted_houses]

# Mostrar algunas predicciones
print("\nPrimeras 5 predicciones:")
for i in range(5):
    print(f"Estudiante {i+1}:")
    print(f"Casa predicha: {predictions[i]}")
    print(f"Probabilidades: Gryffindor: {probabilities[i][0]:.3f}, Hufflepuff: {probabilities[i][1]:.3f}, "
          f"Ravenclaw: {probabilities[i][2]:.3f}, Slytherin: {probabilities[i][3]:.3f}\n")


Primeras 5 predicciones:
Estudiante 1:
Casa predicha: Hufflepuff
Probabilidades: Gryffindor: 0.007, Hufflepuff: 0.985, Ravenclaw: 0.007, Slytherin: 0.001

Estudiante 2:
Casa predicha: Ravenclaw
Probabilidades: Gryffindor: 0.012, Hufflepuff: 0.013, Ravenclaw: 0.938, Slytherin: 0.037

Estudiante 3:
Casa predicha: Gryffindor
Probabilidades: Gryffindor: 0.997, Hufflepuff: 0.001, Ravenclaw: 0.001, Slytherin: 0.002

Estudiante 4:
Casa predicha: Hufflepuff
Probabilidades: Gryffindor: 0.007, Hufflepuff: 0.966, Ravenclaw: 0.022, Slytherin: 0.005

Estudiante 5:
Casa predicha: Hufflepuff
Probabilidades: Gryffindor: 0.025, Hufflepuff: 0.931, Ravenclaw: 0.033, Slytherin: 0.010



In [34]:
# 5. Crear y guardar el archivo houses.csv
predictions_df = pd.DataFrame({
    'Index': range(len(predictions)),
    'Hogwarts House': predictions
})
predictions_df.to_csv('houses.csv', index=False)

# 6. Verificar algunas predicciones
print("\nPrimeras 6 predicciones:")
print(predictions_df.head(6))

# 7. Verificar el formato del archivo creado
print("\nContenido del archivo houses.csv (primeras 7 líneas):")
with open('houses.csv', 'r') as file:
    for i, line in enumerate(file):
        if i < 7:
            print(line.strip())  # strip() elimina los saltos de línea extra


Primeras 6 predicciones:
   Index Hogwarts House
0      0     Hufflepuff
1      1      Ravenclaw
2      2     Gryffindor
3      3     Hufflepuff
4      4     Hufflepuff
5      5      Slytherin

Contenido del archivo houses.csv (primeras 7 líneas):
Index,Hogwarts House
0,Hufflepuff
1,Ravenclaw
2,Gryffindor
3,Hufflepuff
4,Hufflepuff
5,Slytherin
